In [1]:
import torch
import tiktoken

In [2]:
#example of how token ids are converted to embedding vector
input_ids=torch.tensor([2,3,5,1])

In [3]:
#using small vocab size and output_dim=3 dimensions for embedding vector
vocab_size=6 
output_dim=3

torch.manual_seed(123)
embedding_layer=torch.nn.Embedding(vocab_size,output_dim)#torch.nn.Embedding creates a dictionary for embedding vectors with random values (here the embedding layer weight matrix shape = 6 x 3)

print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [4]:
#getting vector embedding weight for specific id
print(embedding_layer(torch.tensor([3])) )

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [5]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [6]:
#implementing Data Loader using pytorch
import torch
from torch.utils.data import Dataset,DataLoader
class GPTDatasetV1(Dataset):

    def __init__(self,text,tokenizer,max_length,stride):
        #text-dataset,tokenizer-gpt tokenizer from tiktoken(BPE tokenizer),max_length-context size,stride-tells how much tokens should the window have for every iteration(eg: if it is 4 ,for every iteration,every window will have 4 tokens)
        self.input_ids=[]
        #token ids of input tokens for entire data as tensors
        self.target_ids=[]
        #token ids of output tokens for entire data as tensors

        token_ids=tokenizer.encode(text,allowed_special={"<|endoftext|>"})
        #encoding whole data

        #looping using sliding window over whole dataset with a context size(max_length)
        #each iteration creates a input_chunk and its corresponding output chunk
        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk=token_ids[i:i+max_length]
            target_chunk=token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]
    #returns the specifix index's input context ids and output context ids



  

In [7]:
def create_dataloader_v1(text,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):

    tokenizer=tiktoken.get_encoding("gpt2")

    #Create dataset
    dataset=GPTDatasetV1(text,tokenizer,max_length,stride)

    #Create Dataloader-dataloader calls __getitem__ function in Dataset class above to create input,output pairs
    dataloader=DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader


In [8]:
#using gpt-2 vocab size and higher dimensional space

vocab_size=50527
output_dim=256

token_embedding_layer=torch.nn.Embedding(vocab_size,output_dim)

In [9]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text=f.read()

In [10]:
max_length=4
dataloader=create_dataloader_v1(
    raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False
)

data_iter=iter(dataloader)
inputs,targets=next(data_iter)



In [11]:
print("Inputs:\n",inputs)
print("\nInputs shape:\n",inputs.shape)
print("\nTargets:\n",targets)
#each row is abatch of token id for 4 different token

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [ ]:
token_embedding=token_embedding_layer(inputs)
print(token_embedding.shape)
#8 is the batch size of inputs, 4 is the number of input tokens at a time(context size), 256 is the number of dimensions for each token id

torch.Size([8, 4, 256])


In [13]:
#positional embedding- 4 tokens are positionally embedded at one time after vector embedding
#for gpt model's absolute embedding approach,need to create another embedding layer

context_length=max_length
pos_embedding_layer=torch.nn.Embedding(context_length,output_dim)


In [14]:
#same positional embedding vector used for 1st input of tokens(4 tokens-context size) from 1st batch is used for all the other 7 inputs of tokens from the 1st batch
pos_embeddings=pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [15]:
#input embeddings-token/vector embeddings+positional embeddings
input_embeddings=token_embedding+pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
